In [13]:
import random
import numpy as np
from copulas.multivariate import Multivariate
import pandas as pd
import matplotlib.pyplot as plt
import trading_vix
from scipy.stats import expon

In [14]:
free_parameters = \
{'cov_para_1': -0.06949259947307429, 'cov_para_2': 0.23929418034836003, 'cov_para_3': 0.39441937092703283, 'cov_para_4': 0.5122625089816987, 'cov_para_5': 0.04957641118779821, 'cov_para_6': -0.17890348702479064, 'beta_1a': 7.779402292936231, 'beta_1b': 15.99508024054407, 'beta_2a': 10.415067866085819, 'beta_2b': 12.093051778272159, 'beta_3a': 7.509928267075036, 'beta_3b': 14.275439706895245, 'beta_4a': 1.3921558019510774, 'beta_4b': 7.916125737682833, 'lambda_expon_1': 0.059660893731805935, 'lambda_expon_2': 0.04483797185269005, 'lambda_expon_3': 0.052272870139135216}

In [15]:
free_para1 = free_parameters['cov_para_1']
free_para2 = free_parameters['cov_para_2']
free_para3 = free_parameters['cov_para_3']
free_para4 = free_parameters['cov_para_4']
free_para5 = free_parameters['cov_para_5']
free_para6 = free_parameters['cov_para_6']

diag_1 = 1
diag_2 = np.sqrt(1-free_para1**2)
if (1-free_para2**2-free_para3**2)<0:
    diag_3 = np.sqrt(np.abs(1-free_para2**2-free_para3**2))
else:
    diag_3 = np.sqrt(1-free_para2**2-free_para3**2)

if (1-free_para4**2-free_para5**2-free_para6**2)<0:
    diag_4 = np.sqrt(np.abs(1-free_para4**2-free_para5**2-free_para6**2))
else:
    diag_4 = np.sqrt(1-free_para4**2-free_para5**2-free_para6**2)

lower_triangular_matrix = np.asarray([[diag_1,0,0,0],
                        [free_para1,diag_2,0,0],
                        [free_para2,free_para3,diag_3,0],
                        [free_para4,free_para5,free_para6,diag_4]])


cov_matrix = lower_triangular_matrix@lower_triangular_matrix.transpose()


#now, define the marginal distribution of the gaussian copula
univerates = [{'loc': 0,
'scale': 1,
'a': free_parameters['beta_1a'],
'b': free_parameters['beta_1b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_2a'],
'b': free_parameters['beta_2b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_3a'],
'b': free_parameters['beta_3b'],
'type': 'copulas.univariate.beta.BetaUnivariate'},
{'loc': 0,
'scale': 1,
'a': free_parameters['beta_4a'],
'b': free_parameters['beta_4b'],
'type': 'copulas.univariate.beta.BetaUnivariate'}]


#now, we construct the gaussian copula
copula_parameters = {}
copula_parameters['covariance'] = cov_matrix
copula_parameters['univariates'] = univerates
copula_parameters['type'] = 'copulas.multivariate.gaussian.GaussianMultivariate'
copula_parameters['columns'] = [0,1,2,3]

new_dist = Multivariate.from_dict(copula_parameters)

In [16]:
#other parameters needed for transforming the features
lambda_expon_1 = free_parameters['lambda_expon_1']
lambda_expon_2 = free_parameters['lambda_expon_2']
lambda_expon_3 = free_parameters['lambda_expon_3']
lambda_expons = [lambda_expon_1,lambda_expon_2,lambda_expon_3]

In [17]:
locol_env = trading_vix.trading_vix()
this_trajectory_reward = []
has_at_least_sell = False
null_objective = True
current_feature = locol_env.reset()

In [18]:
transformed_features = []
for feature_index in range(len(lambda_expons)):
    transformation = expon.cdf(current_feature[feature_index,0],scale = 1.0/lambda_expons[feature_index])
    min_transformation = 0.1
    transformation = min_transformation*np.exp(np.log(1.0/min_transformation)*transformation)
    transformed_features.append(transformation)
transformed_features = np.asarray(transformed_features)
transformed_features = np.reshape(transformed_features,(1,-1))
#holding_position = expit(current_feature[-1,:][0])
holding_position = current_feature[-1,:][0]
if holding_position<0:
    print('holding is less than 0, there is some problem and the holding position is',holding_position)
if holding_position>1:
    print('holding is greater than 1, there is some problem and the holding position is',holding_position)
min_transformed_holding = 0.1
transformed_holding = min_transformed_holding*np.exp(np.log(1.0/min_transformed_holding)*holding_position)
transformed_holding = np.reshape(transformed_holding,(1,1))
data_point_for_df = np.concatenate((transformed_features,transformed_holding),axis = 1)

assert data_point_for_df.shape[1] == 4
data_point_for_copula = pd.DataFrame(data_point_for_df)
action = new_dist.cdf(data_point_for_copula)